## Integrantes do grupo:
- Victor Hugo Ciurlino
- Euripedes Antônio da Silva Junior
- Ricardo Nascimento de Souza
- Danúbia Carvalho Gomes Cantanhede
- Pedro Henrique Ferminío Britto

# importação das bibliotecas:

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

# Iniciação das funções

In [156]:
def ler_dados(dataset_path):
    dados = pd.read_excel(dataset_path)

    dados = dados.iloc[:,1:]

    x = dados.iloc[:,:-1].values
    y = dados.iloc[:,-1].values

    return x, y, dados

def gera_base_treino_teste(dataset_path):

    x, y, dados = ler_dados(dataset_path)

    dados_embaralhados = dados.sample(frac=1,random_state=54321)

    x_treino = dados_embaralhados.iloc[:1297,:-1].values
    y_treino = dados_embaralhados.iloc[:1297,-1].values

    x_teste = dados_embaralhados.iloc[1297:,:-1].values
    y_teste = dados_embaralhados.iloc[1297:,-1].values

    return x_treino, x_teste, y_treino, y_teste


def avalia_parametros(modelo, param_grid, x_treino, y_treino):
    grid_search = GridSearchCV(modelo, param_grid, cv=5, return_train_score=True)
    
    # Ajustando o modelo com os dados de treinamento
    grid_search.fit(x_treino, y_treino)

    # Imprimindo os resultados da busca em grade
    print("Melhores parâmetros: {}".format(grid_search.best_params_))
    print("Melhor pontuação de validação cruzada: {:.2f}".format(grid_search.best_score_))
    k = grid_search.best_params_['n_neighbors']
    weights = grid_search.best_params_['weights']

    return grid_search.best_params_

def gera_tabela_metricas(y_teste, y_pred):
    accuracy = accuracy_score(y_teste, y_pred)
    # matriz_de_confusao = confusion_matrix(y_teste, y_pred)
    precision, recall, fscore, support = precision_recall_fscore_support(y_teste, y_pred)
    df_report = pd.DataFrame({'precision': precision, 'recall': recall, 'fscore': fscore})

    return accuracy, df_report

def predict_models(models, lista_parametros, x_treino, y_treino, x_teste, y_teste):
    
    # List to store the predictions of each model
    lista_configuracao = []
    # Iterate over the models
    for model, hyperparameter in zip(models, lista_parametros):
        print("Novo modelo:")
        # Create a grid search object to find the best hyperparameters for the current model
        clf = GridSearchCV(model, hyperparameter, cv=5, return_train_score=True)

        # Fit the grid search object to the training data
        clf.fit(x_treino, y_treino)
        print("Melhores parâmetros: {}".format(clf.best_params_))
        print("Melhor pontuação de validação cruzada: {:.2f}".format(clf.best_score_))
        # Make predictions on the test data using the best model found by the grid search
        y_pred = clf.predict(x_teste)

        # Append the predictions to the list
        

        accuracy, df_report = gera_tabela_metricas(y_teste, y_pred)
        print(f"Acurácia: {accuracy}")
        # print(df_report)
        lista_configuracao.append({'parametros': clf.best_params_,
                                    'validacao_cruzada': clf.best_score_,
                                    'accuracy': accuracy,
                                    'dataframe': df_report})
    # Return the list of predictions
    return lista_configuracao


# Criação das variáveis para avaliação dos modelos

In [1]:
dataset_path = 'datasets/Digits.xlsx'
lista_modelos = [KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(), SVC(), LogisticRegression()]
lista_parametros = [
{'n_neighbors': [i for i in range(1, 31)],
 'weights': ['uniform', 'distance']
 },
{'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
 'min_samples_split': [2, 4, 6, 8, 10 ,12, 14, 16, 18, 20],
 'criterion': ['gini', 'entropy']
    },
{'n_estimators': [10, 50, 100],
 'max_depth': [2, 4, 6],
 'min_samples_split': [2, 4, 6],
 'min_samples_leaf': [1, 2, 3],
    },
{'C': [0.1, 1.0, 10.0],
'kernel': ['linear', 'rbf', 'poly'],
'gamma': ['scale', 'auto']
},
{'penalty': ['l2'],
 'C': [0.1, 1.0],
 'solver': ['newton-cg']
}
]

NameError: name 'KNeighborsClassifier' is not defined

# Definição dos melhores parâmetros e avaliação dos modelos

In [ ]:
x_treino, x_teste, y_treino, y_teste = gera_base_treino_teste(dataset_path)
lista_predicoes = predict_models(lista_modelos, lista_parametros, x_treino, y_treino, x_teste, y_teste)

In [166]:
lista_predicoes = predict_models(lista_modelos, lista_parametros, x_treino, y_treino, x_teste, y_teste)

Novo modelo:
Melhores parâmetros: {'n_neighbors': 3, 'weights': 'uniform'}
Melhor pontuação de validação cruzada: 0.99
Acurácia: 0.988
Novo modelo:
Melhores parâmetros: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2}
Melhor pontuação de validação cruzada: 0.85
Acurácia: 0.846
Novo modelo:
Melhores parâmetros: {'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 100}
Melhor pontuação de validação cruzada: 0.96
Acurácia: 0.926
Novo modelo:
Melhores parâmetros: {'C': 10.0, 'gamma': 'scale', 'kernel': 'rbf'}
Melhor pontuação de validação cruzada: 0.99
Acurácia: 0.992
Novo modelo:
Melhores parâmetros: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
Melhor pontuação de validação cruzada: 0.96
Acurácia: 0.976


Avaliando a acurácia dos modelos, fica claro que os modelos que melhor performam são: LogisticRegression e KNeighborsClassifier, com a melhor combinação de hiperparâmetros dado a lista criada que realiza uma validação cruzada, avaliando todas as combinações possíveis contidas no dicionário de cada modelo. A combinação de hiperparâmetros que resultou no melhor resultado foi:
- LogisticRegression
    - 'C': 10.0,
    - 'gamma': 'scale',
    - 'kernel': 'rbf'
- KNeighborsClassifier
    - 'n_neighbors': 3,
    - 'weights': 'uniform'

resultando em uma acurácia de 0.99, superando a RandomForestClassifier (0.96) e LogisticRegression(0.96)